# CNN으로 MNIST 분류하기
원본 : https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py

In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 12

# keras.backend 탐구

In [3]:
# keras.backend 확인
keras.backend.backend() 

u'tensorflow'

In [4]:
# keras.backend의 이미지 포맷 확인
keras.backend.image_data_format()

'channels_last'

In [5]:
from keras import backend as K

# 입력 이미지 처리

In [6]:
img_rows, img_cols = 28, 28

### MNIST 이미지 불러오기

In [7]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [8]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


### 이미지에 채널 추가해 주기

In [9]:
if K.image_data_format() == 'channels_first': # Theano인 경우
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_row, img_cols)
    input_shape = (1, img_rows, img_cols)
else: # Tensorflow인 경우
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

### 이미지 전처리해주기

In [10]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [11]:
x_train /= 255
x_test /= 255

In [12]:
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


### 출력 데이터의 차원 맞춰주기

In [14]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [15]:
print(y_train.shape)
print(y_test.shape)

(60000, 10)
(10000, 10)


### 데이터 너무 많아서 줄여주기 (CPU로 돌리다보니...) (내가 추가한 내용)

In [22]:
num_training = 600
num_test = 100

mask_training = list(range(num_training))
mask_test = list(range(num_test))

x_train = x_train[mask_training] # (600, 28, 28, 1)
y_train = y_train[mask_training] # (600, 10)

x_test = x_test[mask_test] # (100, 28, 28, 1)
y_test = y_test[mask_test] # (100, 10)

# 모델 만들기

In [25]:
model = Sequential()

In [26]:
model.add(Conv2D(32, (3,3), 
                 activation='relu', 
                 input_shape=input_shape))
model.add(Conv2D(64, (3,3),
                 activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
__________

# 컴파일하기

In [28]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# 학습시키기

In [34]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 600 samples, validate on 100 samples
Epoch 1/12
600/600 [==============================] - 2s - loss: 0.3474 - acc: 0.8883 - val_loss: 0.3711 - val_acc: 0.8500
Epoch 2/12
600/600 [==============================] - 2s - loss: 0.3010 - acc: 0.9100 - val_loss: 0.3736 - val_acc: 0.9100
Epoch 3/12
600/600 [==============================] - 2s - loss: 0.3252 - acc: 0.9083 - val_loss: 0.2990 - val_acc: 0.9100

# Score 구해주기

In [35]:
score = model.evaluate(x_test, y_test, verbose=1)

 96/100 [===========================>..] - ETA: 0s

In [36]:
print("Test loss:", score[0])
print("Test accuracy", score[1])

Test loss: 0.224203935266
Test accuracy 0.92
